In [1]:
import os
import sys

spark_path = '/Users/pradmishra/Downloads/spark-2.1.0-bin-hadoop2.7'
os.environ['SPARK_HOME']= spark_path
os.environ['HADOOP_HOME']=spark_path
sys.path.append(spark_path+'/bin')
sys.path.append(spark_path+'/python')
sys.path.append(spark_path+'/python/pyspark')
sys.path.append(spark_path+'/python/lib')
sys.path.append(spark_path+'/python/lib/pyspark.zip')
sys.path.append(spark_path+'/python/lib/py4j-0.10.4-src.zip')

In [4]:
import pandas as pd
seeds_dataset = pd.read_csv('https://raw.githubusercontent.com/lettergram/PCA/master/seedAnalysis/seeds_dataset.csv',header=None) 

In [5]:
seeds_dataset.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [7]:
seeds_dataset.columns = ['area','perimeter','compactness','length_of_kernel',
                        'width_of_kernel','asymmetry_coefficient',
                        'length_of_groove','class']

In [8]:
del seeds_dataset['class']

In [11]:
seeds_dataset.to_csv('seeds_dataset.csv',index=None)

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('cluster').getOrCreate()
df = spark.read.csv('seeds_dataset.csv', inferSchema=True, header=True)
df.show(3)

+-----+---------+-----------+----------------+---------------+---------------------+----------------+
| area|perimeter|compactness|length_of_kernel|width_of_kernel|asymmetry_coefficient|length_of_groove|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+
|15.26|    14.84|      0.871|           5.763|          3.312|                2.221|            5.22|
|14.88|    14.57|     0.8811|           5.554|          3.333|                1.018|           4.956|
|14.29|    14.09|      0.905|           5.291|          3.337|                2.699|           4.825|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+
only showing top 3 rows



In [13]:
df.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

assembler = VectorAssembler(inputCols = df.columns, outputCol = 'features')
final_df = assembler.transform(df)
final_df.show(3)

+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+
| area|perimeter|compactness|length_of_kernel|width_of_kernel|asymmetry_coefficient|length_of_groove|            features|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+
|15.26|    14.84|      0.871|           5.763|          3.312|                2.221|            5.22|[15.26,14.84,0.87...|
|14.88|    14.57|     0.8811|           5.554|          3.333|                1.018|           4.956|[14.88,14.57,0.88...|
|14.29|    14.09|      0.905|           5.291|          3.337|                2.699|           4.825|[14.29,14.09,0.90...|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+
only showing top 3 rows



In [15]:
final_df.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')
scaler_model = scaler.fit(final_df)
final_df = scaler_model.transform(final_df)
final_df.show(3)

+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+--------------------+
| area|perimeter|compactness|length_of_kernel|width_of_kernel|asymmetry_coefficient|length_of_groove|            features|      scaledFeatures|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+--------------------+
|15.26|    14.84|      0.871|           5.763|          3.312|                2.221|            5.22|[15.26,14.84,0.87...|[5.24452795332028...|
|14.88|    14.57|     0.8811|           5.554|          3.333|                1.018|           4.956|[14.88,14.57,0.88...|[5.11393027165175...|
|14.29|    14.09|      0.905|           5.291|          3.337|                2.699|           4.825|[14.29,14.09,0.90...|[4.91116018695588...|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+--------------

In [17]:
final_df.take(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [18]:
kmeans = KMeans(featuresCol = 'scaledFeatures', k=3)
model = kmeans.fit(final_df)

In [19]:
print('WSSSE:', model.computeCost(final_df))

('WSSSE:', 429.0755867054652)


In [20]:
centers = model.clusterCenters()
print(centers)

[array([  6.31670546,  12.37109759,  37.39491396,  13.91155062,
         9.748067  ,   2.39850262,  12.2661748 ]), array([  4.87257659,  10.88120146,  37.27692543,  12.3410157 ,
         8.55443412,   1.81649411,  10.32998598]), array([  4.06105916,  10.13979506,  35.80536984,  11.82133095,
         7.50395937,   3.27185132,  10.42126018])]


In [21]:
model.transform(final_df).select('scaledFeatures', 'prediction').show()

+--------------------+----------+
|      scaledFeatures|prediction|
+--------------------+----------+
|[5.24452795332028...|         1|
|[5.11393027165175...|         1|
|[4.91116018695588...|         1|
|[4.75650503761158...|         1|
|[5.54696468981581...|         1|
|[4.94209121682475...|         1|
|[5.04863143081749...|         1|
|[4.84929812721816...|         1|
|[5.71536696354628...|         0|
|[5.65006812271202...|         0|
|[5.24452795332028...|         1|
|[4.82180387844584...|         1|
|[4.77368894309428...|         1|
|[4.73588435103234...|         1|
|[4.72213722664617...|         1|
|[5.01426361985209...|         1|
|[4.80805675405968...|         1|
|[5.39230954047151...|         1|
|[5.05206821191403...|         1|
|[4.37158555479908...|         2|
+--------------------+----------+
only showing top 20 rows

